In [ ]:
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

In [ ]:
import numpy as np

In [ ]:
from sklearn.cross_validation import train_test_split

from numpy import newaxis

In [ ]:
data = np.load("./data/training_data.npy")

## Data Preprocessing and Reshaping

In [ ]:
X = [x[0] for x in data]
y = [1 if y[1][0] == 1 else 0 for y in data]

In [ ]:
X_cnn = [x[:,:,newaxis] for x in X]
X_cnn= np.array(X_cnn)
y = np.array(y)

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X_cnn,y)

## SVM Model

In [ ]:
# from sklearn.svm import SVC
# from sklearn.grid_search import GridSearchCV

# svm = SVC()

# parameter_candidates = {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']}
# clf = GridSearchCV(estimator=svm, param_grid=parameter_candidates)

In [ ]:
# X_svm = [x.ravel() for x in X]

In [ ]:
# X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X_svm,y)

In [ ]:
# X_train_svm = np.array(X_train_svm)
# X_test_svm = np.array(X_test_svm)

In [ ]:
# y_train_svm = np.array(y_train_svm)
# y_test_svm = np.array(y_test_svm)

In [ ]:
# clf.fit(X_train_svm, y_train_svm)

## Baseline Model

In [ ]:
def create_baseline():
    optimizer = 'adam'
    objective = 'binary_crossentropy'

    model = Sequential()

    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(224, 224, 1), activation='relu'))
    model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    # model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))

    # model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    # model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))

    # model.add(Flatten())
    # model.add(Dense(256, activation='relu'))
    # model.add(Dropout(0.5))

    # model.add(Dense(256, activation='relu'))
    # model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [ ]:
model = create_baseline()

In [ ]:
model.summary()

In [ ]:
## Callback for loss logging per epoch
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

In [ ]:
epochs = 10
history = LossHistory()
model.fit(X_train, y_train, validation_split=0.25, nb_epoch=epochs, batch_size=32, verbose=2, shuffle=True, callbacks=[history])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
predictions = model.predict(X_test)

In [ ]:
baseline_accuracy = accuracy_score(y_test, predictions)

In [ ]:
print("The accuracy of baseline model is: " + str(baseline_accuracy))

In [ ]:
model.save("baseline_cnn_2_layers.h5")

In [ ]:
baseline_val_losses = np.array(history.val_losses)
baseline_losses = np.array(history.losses)

In [ ]:
np.save("baseline_val_losses.npy", baseline_val_losses)
np.save("baseline_losses.npy", baseline_losses)

In [ ]:
baseline_val_losses

## Creating a little deeper model

In [ ]:
def create_grayscale_deeper_model():
    optimizer = 'adam'
    objective = 'binary_crossentropy'

    model = Sequential()

    model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=(224, 224, 1), activation='relu'))
    model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    # model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2, 2)))

    # model.add(Flatten())
    # model.add(Dense(256, activation='relu'))
    # model.add(Dropout(0.5))

    # model.add(Dense(256, activation='relu'))
    # model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [ ]:
grayscale_deeper_model = create_grayscale_deeper_model()

In [ ]:
grayscale_deeper_model.summary()

In [ ]:
epochs = 15
grayscale_deeper_history = LossHistory()
grayscale_deeper_model.fit(X_train, y_train, validation_split=0.25, nb_epoch=epochs, batch_size=32, verbose=2, shuffle=True, callbacks=[grayscale_deeper_history])

In [ ]:
predictions_7_conv = grayscale_deeper_model.predict(X_test)

In [ ]:
grayscale_7_conv_acc = accuracy_score(y_test, predictions)
print("The accuracy of grayscale model with 7 convolution layers is: " + str(grayscale_7_conv_acc))